<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image
import tensorflow as tf
from tensorflow.data import Dataset
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 11.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/gdrive


In [7]:
from sklearn.preprocessing import LabelEncoder

# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = '/content/'
imgtrain_dir = data_path + "train/images/"
csvtrain_file = data_path + "train.csv"

imgtest_dir = data_path + "test/images/"
csvtest_file = data_path + "test.csv"
# Leer el fichero CSV con las etiquetas

df = pd.read_csv(csvtrain_file, dtype = {"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype = {"class": "category"})
# Codificar las etiquetas utilizando LabelEncoder
label_encoder = LabelEncoder()
df['class_encoded'] = label_encoder.fit_transform(df['class'])

# Convertir las etiquetas codificadas en un vector one-hot
class_one_hot = pd.get_dummies(df['class_encoded'], prefix='class')

# Renombrar las columnas del vector one-hot
class_one_hot.columns = ['normal', 'benign', 'malignant']

# Concatenar el DataFrame original con las etiquetas one-hot
df = pd.concat([df[['image_filename']], class_one_hot], axis=1)

# Imprimir las primeras filas del DataFrame para verificar
print(df.head())


  image_filename  normal  benign  malignant
0  image_003.png    True   False      False
1  image_004.png    True   False      False
2  image_008.png    True   False      False
3  image_012.png    True   False      False
4  image_016.png    True   False      False


In [9]:
# Dividir el conjunto en entrenamiento, validación y test (80:10:10)
val_size = int(len(df) * 0.15)
test_size = int(len(df) * 0.15)

df = df.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dftest = df[:test_size]
dfval = df[test_size:test_size+val_size]
dftrain = df[test_size+val_size:]

print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)
dftest = dftest.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 237
Número de ejemplos del conjunto de validación: 50
Número de ejemplos del conjunto de test: 50


In [10]:
dftrain

,image_filename,normal,benign,malignant
0,image_056.png,True,False,False
1,image_267.png,False,False,True
2,image_020.png,True,False,False
3,image_131.png,False,True,False
4,image_285.png,False,True,False
...,...,...,...,...
232,image_372.png,True,False,False
233,image_115.png,False,True,False
234,image_158.png,True,False,False
235,image_161.png,False,False,True


In [ ]:
archivos = os.listdir(imgtrain_dir)
anchuras = []
alturas = []
for archivo in archivos:
      imagen = Image.open(os.path.join(imgtrain_dir, archivo))
      ancho, alto = imagen.size
      anchuras.append(ancho)
      alturas.append(alto)
media_anchura = sum(anchuras) / len(anchuras)
media_altura = sum(alturas) / len(alturas)

print("Media de anchura:", media_anchura)
print("Media de altura:", media_altura)

Media de anchura: 634.9139465875371
Media de altura: 514.9317507418398


In [29]:
import tensorflow as tf
from tensorflow.data import Dataset

# Dimensiones deseadas de la imagen
img_width, img_height = 224,224
n_channels = 3                # número de canales (RGB)
n_classes = 3                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = ['normal', 'benign', 'malignant']  # lista de nombres de las columnas de las etiquetas

# Cargar y preprocesar imágenes
def load_and_preprocess_image(image_filename, label_one_hot):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    # Redimensionar la imagen al tamaño deseado con relleno de ceros si es necesario
    image = tf.image.resize_with_pad(image, img_width, img_height)
    image = image / 255.0                               # normalización
    label = label_one_hot                                # Utilizar etiquetas codificadas como vector one-hot
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_filenames = df[x_col].values
    labels = df[y_col].values
    dataset = tf.data.Dataset.from_tensor_slices((image_filenames, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 32
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)
test_dataset = get_dataset(dftest).batch(batch_size)
print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')
print(f'Número de lotes del conjunto de test: {len(test_dataset)}')

Número de lotes del conjunto de entrenamiento: 8
Número de lotes del conjunto de validación: 2
Número de lotes del conjunto de test: 2


In [14]:
pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 725.4 kB/s eta 0:00:00


In [41]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from efficientnet.tfkeras import EfficientNetB0

def build_efficientnet(input_shape, num_classes):
    # Definir el modelo EfficientNetB0 pre-entrenado
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)

    # Congelar las capas de la base del modelo
    base_model.trainable = False

    # Agregar capas adicionales de clasificación
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)

    # Combinar el modelo base con las capas adicionales
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Ejemplo de uso:
input_shape = (img_width, img_height, 3)  # Tamaño de entrada de las imágenes
num_classes = 3  # Número de clases en ImageNet
model = build_efficientnet(input_shape, num_classes)

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 stem_conv (Conv2D)          (None, 112, 112, 32)         864       ['input_10[0][0]']            
                                                                                                  
 stem_bn (BatchNormalizatio  (None, 112, 112, 32)         128       ['stem_conv[0][0]']           
 n)                                                                                               
                                                                                                  
 stem_activation (Activatio  (None, 112, 112, 32)         0         ['stem_bn[0][0]']       

In [42]:
import numpy as np

# Entrenar el modelo con los datos preparados previamente
history = model.fit(train_dataset,
          epochs=20,   # número de epochs
          verbose=2,  # muestra información al finalizar cada epoch
          validation_data=val_dataset)

# Imprimir el error mínimo de entrenamiento y validación
train_trace = np.array(history.history['loss'])
print(f'\nError mínimo en entrenamiento: {min(train_trace):.6f}')

val_trace = np.array(history.history['val_loss'])
print(f'Error mínimo en validación: {min(val_trace):.6f}')

Epoch 1/20
8/8 - 9s - loss: 1.0846 - accuracy: 0.5612 - val_loss: 0.8231 - val_accuracy: 0.6200 - 9s/epoch - 1s/step
Epoch 2/20
8/8 - 1s - loss: 0.6939 - accuracy: 0.7215 - val_loss: 0.7325 - val_accuracy: 0.6200 - 1s/epoch - 156ms/step
Epoch 3/20
8/8 - 1s - loss: 0.6498 - accuracy: 0.6793 - val_loss: 0.6101 - val_accuracy: 0.7400 - 1s/epoch - 155ms/step
Epoch 4/20
8/8 - 1s - loss: 0.5801 - accuracy: 0.7173 - val_loss: 0.5663 - val_accuracy: 0.8000 - 1s/epoch - 160ms/step
Epoch 5/20
8/8 - 2s - loss: 0.5133 - accuracy: 0.7637 - val_loss: 0.5786 - val_accuracy: 0.7600 - 2s/epoch - 278ms/step
Epoch 6/20
8/8 - 1s - loss: 0.4102 - accuracy: 0.8565 - val_loss: 0.5760 - val_accuracy: 0.7800 - 1s/epoch - 156ms/step
Epoch 7/20
8/8 - 1s - loss: 0.3698 - accuracy: 0.8903 - val_loss: 0.5639 - val_accuracy: 0.7600 - 1s/epoch - 158ms/step
Epoch 8/20
8/8 - 1s - loss: 0.3829 - accuracy: 0.8397 - val_loss: 0.5373 - val_accuracy: 0.7800 - 1s/epoch - 157ms/step
Epoch 9/20
8/8 - 1s - loss: 0.3467 - accura

In [43]:
test_loss, test_medida = model.evaluate(test_dataset, verbose=2)
print("test_loss: %.4f, test_medida: %.4f" % (test_loss, test_medida))

2/2 - 0s - loss: 0.5317 - accuracy: 0.8000 - 414ms/epoch - 207ms/step
test_loss: 0.5317, test_medida: 0.8000


In [44]:
# Directorio donde se encuentran las imágenes de prueba
test_images_dir = 'test/images/'

# Obtener la lista de nombres de archivos de las imágenes de prueba
test_filenames = os.listdir(test_images_dir)

# Crear un DataFrame para almacenar las predicciones
predictions_df = pd.DataFrame({'image_filename': test_filenames})

# Crear columnas para 'normal', 'benign' y 'malignant' con valores iniciales de 0
predictions_df['normal'] = 0
predictions_df['benign'] = 0
predictions_df['malignant'] = 0

# Iterar sobre cada imagen de prueba
for filename in test_filenames:
    # Cargar la imagen y preprocesarla
    img_path = os.path.join(test_images_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión adicional para el lote

    # Realizar la predicción
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha

    # Actualizar las columnas correspondientes según la predicción
    if predicted_class == 0:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'normal'] = 1
    elif predicted_class == 1:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'benign'] = 1
    elif predicted_class == 2:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'malignant'] = 1

# Ordenar el DataFrame por el nombre del archivo de imagen
predictions_df = predictions_df.sort_values(by='image_filename')

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('test_predictions.csv', index=False)

# Imprimir las primeras filas del DataFrame para verificar
print(predictions_df.head())


1/1 [==============================] - 0s 26ms/step
    image_filename  normal  benign  malignant
64   image_005.png       0       1          0
53   image_006.png       1       0          0
112  image_007.png       1       0          0
40   image_009.png       1       0          0
21   image_010.png       0       1          0
